In [ ]:
import pandas as pd
import h5py
import numpy as np
import os

def csv_to_h5_and_csv(csv_file, h5_file, string_csv_file, chunk_size=100000):
    """
    將 CSV 檔案轉換為 H5 格式（數值欄位）和 CSV 格式（字串欄位）。

    Args:
        csv_file (str): 輸入的 CSV 檔案路徑。
        h5_file (str): 輸出的 H5 檔案路徑。
        string_csv_file(str): 輸出的字串csv檔案路徑。
        chunk_size (int): 每次讀取的 CSV 行數。
    """
    # 讀取 CSV 檔案，並指定資料類型
    reader = pd.read_csv(csv_file, chunksize=chunk_size)
    string_data = [] #儲存字串資料
    for i, chunk in enumerate(reader):
        if i == 0:
            string_data = chunk[['Title', 'Abstract', 'Authors', 'Journal', 'Keyword']].copy()
            numerical_data = chunk[['PMID', 'Year']].copy()
        else:
            string_data = pd.concat([string_data, chunk[['Title', 'Abstract', 'Authors', 'Journal', 'Keyword']]])
            numerical_data = pd.concat([numerical_data, chunk[['PMID', 'Year']]])
    string_data.to_csv(string_csv_file, index=False)
    with h5py.File(h5_file, 'w') as hf:
        hf.create_dataset('PMID', data=numerical_data['PMID'].values)
        hf.create_dataset('Year', data=numerical_data['Year'].values)

# 使用範例
csv_file_path = "../data/pubmed_baseline/merged_output.csv"
h5_file_path = 'merged_output.h5'
string_csv_file_path = 'merged_output_string.csv'
csv_to_h5_and_csv(csv_file_path, h5_file_path, string_csv_file_path)